# San Francisco Police Deprt. Crime Data Analysis Pipeline

In a dynamic urban environment like San Francisco, analyzing crime data is essential for understanding crime patterns, identifying hotspots, and devising effective strategies to maintain public safety. This dataset provides a wealth of information related to various aspects of reported incidents, including incident date and time, incident category, location, resolution, and more. Exploring this dataset through dashboard will help us gain insights into crime trends, incident distribution, and police response.

The dataset's columns encompass a wide range of information, including incident timestamps, descriptions, categories, resolution status, geographical coordinates, police district, neighborhood, and more. This Analysis project focuses on conducting a comprehensive analysis of the crime dataset based in San Francisco. The dataset contains incident reports spanning from 2018 to 2023, by performing exploratory data analysis, we aim to uncover patterns, correlations, and anomalies within this dataset to support data-driven decision-making for law enforcement and city planning.

In [1]:
#importing necessary libraries
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)

In [2]:
#importing the dataset
df = pd.read_csv('../Data/SF_Crime_data/Combined/Police_Department_Incident_Reports__2018_to_Present_20240106.csv')

In [3]:
#print of the sample of the dataframe
df.head()

,Incident Datetime,Incident Date,Incident Time,Incident Year,Incident Day of Week,Report Datetime,Row ID,Incident ID,Incident Number,CAD Number,Report Type Code,Report Type Description,Filed Online,Incident Code,Incident Category,Incident Subcategory,Incident Description,Resolution,Intersection,CNN,Police District,Analysis Neighborhood,Supervisor District,Supervisor District 2012,Latitude,Longitude,Point,Neighborhoods,ESNCAG - Boundary File,Central Market/Tenderloin Boundary Polygon - Updated,Civic Center Harm Reduction Project Boundary,HSOC Zones as of 2018-06-05,Invest In Neighborhoods (IIN) Areas,Current Supervisor Districts,Current Police Districts
0,2023/03/13 11:41:00 PM,2023/03/13,23:41,2023,Monday,2023/03/13 11:41:00 PM,125373607041,1253736,230167874,NaN,VS,Vehicle Supplement,NaN,7041,Recovered Vehicle,Recovered Vehicle,"Vehicle, Recovered, Auto",Open or Active,NaN,NaN,Out of SF,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2023/03/01 05:02:00 AM,2023/03/01,05:02,2023,Wednesday,2023/03/11 03:40:00 PM,125379506374,1253795,236046151,NaN,II,Coplogic Initial,True,6374,Larceny Theft,Larceny Theft - Other,"Theft, Other Property, >$950",Open or Active,NaN,NaN,Mission,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2023/03/13 01:16:00 PM,2023/03/13,13:16,2023,Monday,2023/03/13 01:17:00 PM,125357107041,1253571,220343896,NaN,VS,Vehicle Supplement,NaN,7041,Recovered Vehicle,Recovered Vehicle,"Vehicle, Recovered, Auto",Open or Active,NaN,NaN,Out of SF,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2023/03/13 10:59:00 AM,2023/03/13,10:59,2023,Monday,2023/03/13 11:00:00 AM,125355107041,1253551,230174885,NaN,VS,Vehicle Supplement,NaN,7041,Recovered Vehicle,Recovered Vehicle,"Vehicle, Recovered, Auto",Open or Active,NaN,NaN,Out of SF,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2023/03/14 06:44:00 PM,2023/03/14,18:44,2023,Tuesday,2023/03/14 06:45:00 PM,125402407041,1254024,230176728,NaN,VS,Vehicle Supplement,NaN,7041,Recovered Vehicle,Recovered Vehicle,"Vehicle, Recovered, Auto",Open or Active,NaN,NaN,Out of SF,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
'''
#Changing names of columns to insert Underscore in spaces.
column_name = df.columns
#Init of an empty list
column_name_changed = list()
#Init of a for loop to itterate through column names
for i in range(len(column_name)):
    #add the underscored name to the list
    column_name_changed.append(column_name[i].replace(" ","_"))
#substituting colums names to the new ones
df.columns = column_name_changed
'''

#Changing names of columns to insert Underscore in spaces
df.rename(columns=lambda x: x.strip().replace(" ","_"),inplace=True)

#standardizing the Report code respective to the report type description
#Init of the dictionary
Report_Code_correction = {
    'Vehicle Supplement' : 'VS',
    'Coplogic Initial' : 'CI',
    'Coplogic Supplement' : 'CS',
    'Initial' : 'II',
    'Initial Supplement' : 'IS',
    'Vehicle Initial': 'VI'
}
#mapping the standardized values using the dictionary to dataframe column values
df['Report_Type_Code'] = df['Report_Type_Description'].map(Report_Code_correction)


#to standardize the values in the Incident Category column that represent the same concept
#Init of the dictionary
replace_dict = {
    'Recovered Vehicle': 'Vehicle Theft and Recovery',
    'Motor Vehicle Theft': 'Vehicle Theft and Recovery',
    'Motor Vehicle Theft?': 'Vehicle Theft and Recovery',
    'Recovered Vehicle': 'Vehicle Theft and Recovery',
    'Larceny Theft': 'Property Crimes',
    'Lost Property': 'Property Crimes',
    'Stolen Property': 'Property Crimes',
    'Burglary': 'Property Crimes',
    'Vandalism': 'Property Crimes',
    'Drug Violation': 'Drug-related Incidents',
    'Drug Offense': 'Drug-related Incidents',
    'Non-Criminal': 'Miscellaneous',
    'Case Closure': 'Miscellaneous',
    'Other Miscellaneous': 'Miscellaneous',
    'Other Offenses': 'Miscellaneous',
    'Other': 'Miscellaneous',
    'Miscellaneous Investigation': 'Miscellaneous',
    'Suspicious Occ': 'Suspicious',
    'Suspicious': 'Suspicious',
    'Missing Person': 'Police Interaction Incidents',
    'Courtesy Report': 'Police Interaction Incidents',
    'Fraud': 'Fraudulent Activities',
    'Forgery And Counterfeiting': 'Fraudulent Activities',
    'Prostitution': 'Sex-related Incidents',
    'Human Trafficking, Commercial Sex Acts': 'Sex-related Incidents',
    'Human Trafficking (A), Commercial Sex Acts': 'Sex-related Incidents',
    'Human Trafficking (B), Involuntary Servitude': 'Sex-related Incidents',
    'Embezzlement': 'Financial and Regulatory Offenses',
    'Gambling': 'Financial and Regulatory Offenses',
    'Liquor Laws': 'Financial and Regulatory Offenses'
}
#mapping the standardized values using the dictionary to dataframe column values
df['Incident_Category'].replace(replace_dict, inplace=True)


#To fill the missing values in the Incident Category by using the respective Incident Description
#Init of the dictionary
description_to_incident = {
    'Vehicle, Seizure Order Service': 'Vehicle Theft and Recovery',
    'Gun Violence Restraining Order': 'Weapons Offense',
    'Driving, Stunt Vehicle/Street Racing': 'Traffic Collision',
    'Cryptocurrency Related Crime (secondary code only)': 'Financial and Regulatory Offenses',
    'Auto Impounded': 'Vehicle Impounded',
    'Theft, Boat': 'Property Crimes',
    'Theft, Animal, Att.': 'Property Crimes',
    'SFMTA Muni Transit Operator-Bus/LRV': 'SFMTA',
    'Military Ordinance': 'Weapons Offense',
    'SFMTA Parking and Control Officer': 'SFMTA',
    'Cloned Cellular Phone, Use': 'Fraudulent Activities',
    'Public Health Order Violation, Notification': 'Miscellaneous',
    'Public Health Order Violation, After Notification': 'Miscellaneous',
    'Assault, Commission of While Armed': 'Assault',
    'Theft, Phone Booth, <$50': 'Property Crimes',
    'Gun Violence Restraining Order Violation': 'Weapons Offense',
    'Theft, Phone Booth, $200-$950': 'Property Crimes',
    'Theft, Phone Booth, $50-$200': 'Property Crimes',
    'Vehicle, Seizure Order': 'Vehicle Theft and Recovery',
    'SFMTA Employee-Non Operator/Station Agent-Other Employee': 'SFMTA',
    'Assault, By Police Officers': 'Assault',
    'Crimes Involving Receipts or Titles': 'Fraudulent Activities',
    'Procurement, Pimping, & Pandering': 'Sex-related Incidents',
    'Service of Documents Related to a Civil Drug Abatement and/or Public Nuisance Action': 'Miscellaneous',
    'Pyrotechnic Explosive Device - Barrel Bomb': 'Fire Report',
    'Theft, Phone Booth, >$950': 'Property Crimes'
}
#mapping the standardized values using the dictionary to dataframe column values
df['Incident_Category'] = df['Incident_Category'].fillna(df['Incident_Description'].map(description_to_incident))


#converting date columns to datetime format in dataframe
df['Incident_Datetime']= pd.to_datetime(df['Incident_Datetime'], format= '%Y/%m/%d %I:%M:%S %p')
df['Report_Datetime']= pd.to_datetime(df['Report_Datetime'], format= '%Y/%m/%d %I:%M:%S %p')

#dropping the incomplete month data of September 2023
#df = df.drop((df[(df['Incident_Datetime'].dt.month == 9) & (df['Incident_Year'] == 2023)]).index)

#Converting boolean column to binary column
df['Filed_Online'] = df['Filed_Online'].notnull().astype(int)

In [5]:
#print of the sample of the dataframe
df.head()

,Incident_Datetime,Incident_Date,Incident_Time,Incident_Year,Incident_Day_of_Week,Report_Datetime,Row_ID,Incident_ID,Incident_Number,CAD_Number,Report_Type_Code,Report_Type_Description,Filed_Online,Incident_Code,Incident_Category,Incident_Subcategory,Incident_Description,Resolution,Intersection,CNN,Police_District,Analysis_Neighborhood,Supervisor_District,Supervisor_District_2012,Latitude,Longitude,Point,Neighborhoods,ESNCAG_-_Boundary_File,Central_Market/Tenderloin_Boundary_Polygon_-_Updated,Civic_Center_Harm_Reduction_Project_Boundary,HSOC_Zones_as_of_2018-06-05,Invest_In_Neighborhoods_(IIN)_Areas,Current_Supervisor_Districts,Current_Police_Districts
0,2023-03-13 23:41:00,2023/03/13,23:41,2023,Monday,2023-03-13 23:41:00,125373607041,1253736,230167874,NaN,VS,Vehicle Supplement,0,7041,Vehicle Theft and Recovery,Recovered Vehicle,"Vehicle, Recovered, Auto",Open or Active,NaN,NaN,Out of SF,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2023-03-01 05:02:00,2023/03/01,05:02,2023,Wednesday,2023-03-11 15:40:00,125379506374,1253795,236046151,NaN,CI,Coplogic Initial,1,6374,Property Crimes,Larceny Theft - Other,"Theft, Other Property, >$950",Open or Active,NaN,NaN,Mission,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2023-03-13 13:16:00,2023/03/13,13:16,2023,Monday,2023-03-13 13:17:00,125357107041,1253571,220343896,NaN,VS,Vehicle Supplement,0,7041,Vehicle Theft and Recovery,Recovered Vehicle,"Vehicle, Recovered, Auto",Open or Active,NaN,NaN,Out of SF,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2023-03-13 10:59:00,2023/03/13,10:59,2023,Monday,2023-03-13 11:00:00,125355107041,1253551,230174885,NaN,VS,Vehicle Supplement,0,7041,Vehicle Theft and Recovery,Recovered Vehicle,"Vehicle, Recovered, Auto",Open or Active,NaN,NaN,Out of SF,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2023-03-14 18:44:00,2023/03/14,18:44,2023,Tuesday,2023-03-14 18:45:00,125402407041,1254024,230176728,NaN,VS,Vehicle Supplement,0,7041,Vehicle Theft and Recovery,Recovered Vehicle,"Vehicle, Recovered, Auto",Open or Active,NaN,NaN,Out of SF,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
#Checking the null values in each column
df.isnull().sum()

Incident_Datetime                                            0
Incident_Date                                                0
Incident_Time                                                0
Incident_Year                                                0
Incident_Day_of_Week                                         0
Report_Datetime                                              0
Row_ID                                                       0
Incident_ID                                                  0
Incident_Number                                              0
CAD_Number                                              183941
Report_Type_Code                                             0
Report_Type_Description                                      0
Filed_Online                                                 0
Incident_Code                                                0
Incident_Category                                            6
Incident_Subcategory                                   

In [7]:
df = df[(df['Incident_Datetime'] <= '2023-12-31 23:59:59') & (df['Report_Datetime'] <= '2023-12-31 23:59:59')]

In [8]:
#displaying the min and the max dates of incidents
print("Incident oldest date",min(df['Incident_Datetime']))
print("Incident latest date",max(df['Incident_Datetime']))

Incident oldest date 2018-01-01 00:00:00
Incident latest date 2023-12-31 23:35:00


In [9]:
#displaying the min and the max dates of reports
print("Report oldest date",min(df['Report_Datetime']))
print("Report latest date",max(df['Report_Datetime']))

Report oldest date 2018-01-01 00:01:00
Report latest date 2023-12-31 23:35:00


### Day Part Dimension table

In [10]:
#Init of a dict for day part
day_part_dict = {1: 'Late Night', 2: 'Early Morning', 3: 'Morning', 4: 'Noon', 5: 'Evening', 6: 'Night'}
#Converting dict to dataframe
df_day_part = pd.DataFrame(list(day_part_dict.items()), columns=['Id', 'day_part'])
#print the dataframe
df_day_part

,Id,day_part
0,1,Late Night
1,2,Early Morning
2,3,Morning
3,4,Noon
4,5,Evening
5,6,Night


### Time Dimension table

In [11]:
#Init of a dataframe for the time dimension table
df_time = pd.DataFrame(pd.date_range(start="2000-01-01", end="2000-01-02", freq="T", inclusive='left'))
df_time['Id'] = df_time.index
#To extract various components from the time column
#extract time from datetime column
df_time['Time'] = df_time[0].dt.time
#extract hour from datetime column
df_time['Hour'] = df_time[0].dt.hour
#extract minutes from datetime column
df_time['Minute'] = df_time[0].dt.minute
#calculate the categorical number from the hour column for the day part
df_time['Day_part'] = (df_time['Hour'] % 24 + 4) // 4
#mapping the day part name for the numerical value
df_time['Day_part'] = df_time['Day_part'].replace(day_part_dict)
#mapping the day part name from the dimension table of day part
df_time['Day_part_Id'] = df_time['Day_part'].replace(df_day_part.set_index('day_part')['Id'])


#dropping the unnecessary column
df_time.drop(columns=[0, 'Day_part'], inplace=True)


#df_time['time'] = pd.to_datetime(df_time['time'], format= '%I:%M:%S')
#df_time['time'] = df_time['time'].strftime('%I:%M:%S')

#To extract various tangents from the time columns.
#df_time['Hour'] = df_time['Time'].astype(str).str[:2].astype(int)

In [12]:
#print of the sample of the dataframe
df_time.head()

,Id,Time,Hour,Minute,Day_part_Id
0,0,00:00:00,0,0,1
1,1,00:01:00,0,1,1
2,2,00:02:00,0,2,1
3,3,00:03:00,0,3,1
4,4,00:04:00,0,4,1


### Date Dimension table

In [13]:
#Init of a dataframe for the dimension table
df_date = pd.DataFrame(pd.date_range(start='2018-01-01 00:01:00', end='2023-12-31 23:59:59', freq="D", inclusive='both'))
#creating a primary key column using index of the dataframe
df_date['Id'] = df_date.index
#extracting date from the datetime value
df_date['Date'] = df_date[0].dt.date
#dropping unnecessary columns
df_date.drop(columns=[0], inplace=True)
#to convert the column to date type format
df_date['Date'] = pd.to_datetime(df_date['Date'])
#To exrtact various tangents from the date columns.
df_date['Dayofmonth'] = df_date['Date'].dt.day
df_date['Month_Number'] = df_date['Date'].dt.month
df_date['Month_Name'] = df_date['Date'].dt.month_name()
df_date['Dayofweek_number'] = df_date['Date'].dt.day_of_week
df_date['Dayofweek_name'] = df_date['Date'].dt.day_name()
df_date['Year'] = df_date['Date'].dt.year

In [14]:
#print of the sample of the dataframe
df_date.head()

,Id,Date,Dayofmonth,Month_Number,Month_Name,Dayofweek_number,Dayofweek_name,Year
0,0,2018-01-01,1,1,January,0,Monday,2018
1,1,2018-01-02,2,1,January,1,Tuesday,2018
2,2,2018-01-03,3,1,January,2,Wednesday,2018
3,3,2018-01-04,4,1,January,3,Thursday,2018
4,4,2018-01-05,5,1,January,4,Friday,2018


### Function for Dimensional tables

In [15]:
#Init of a function
#To transform the tables into new dataframe for dimension tables
def dimension_tbl_fun(df_temp):
    #dropping the duplicate rows across the dataframe
    df_temp.drop_duplicates(inplace=True)
    #dropping missing values
    df_temp.dropna(inplace=True)
    #Resetting the index after drop code
    df_temp.reset_index(inplace=True)
    #dropping unnecessary columns
    df_temp.drop(columns=['index'], inplace=True)
    #creating a primary key column using index of the dataframe
    df_temp['Id'] = df_temp.index
    return df_temp

### Report Info Dimension table

In [16]:
#Calling the dimension functions
df_report_info = dimension_tbl_fun(df[['Report_Type_Code','Report_Type_Description']].copy())
#dropping unnecessary columns
df_report_info.drop(columns='Id',inplace=True)

In [17]:
#print of the sample of the dataframe
df_report_info

,Report_Type_Code,Report_Type_Description
0,VS,Vehicle Supplement
1,CI,Coplogic Initial
2,CS,Coplogic Supplement
3,II,Initial
4,IS,Initial Supplement
5,VI,Vehicle Initial


### Incident Info Dimension table

In [18]:
#Init of a dataframe for the dimension table
df_incident_info = df[['Incident_Code','Incident_Category','Incident_Subcategory','Incident_Description']].copy()
#dropping the duplicate rows across the dataframe
df_incident_info.drop_duplicates(inplace=True)
#Dropping the row with duplicate incident code = 7060
#df_incident_info.drop(labels=[283526], inplace=True)
#Resetting the index after drop code
df_incident_info.reset_index(inplace=True)
#dropping unnecessary columns
df_incident_info.drop(columns=['index'], inplace=True)

In [19]:
#print of the sample of the dataframe
df_incident_info.head()

,Incident_Code,Incident_Category,Incident_Subcategory,Incident_Description
0,7041,Vehicle Theft and Recovery,Recovered Vehicle,"Vehicle, Recovered, Auto"
1,6374,Property Crimes,Larceny Theft - Other,"Theft, Other Property, >$950"
2,6372,Property Crimes,Larceny Theft - Other,"Theft, Other Property, $50-$200"
3,6244,Property Crimes,Larceny - From Vehicle,"Theft, From Locked Vehicle, >$950"
4,4134,Assault,Simple Assault,Battery


### Police District Dimension table

In [20]:
#Calling the dimension functions
df_police_district = dimension_tbl_fun(df[['Police_District']].copy())
#setting the order of the columns in dataframe
df_police_district = df_police_district[['Id', 'Police_District']]

In [21]:
#print of the sample of the dataframe
df_police_district.head()

,Id,Police_District
0,0,Out of SF
1,1,Mission
2,2,Central
3,3,Park
4,4,Bayview


### Resolution Dimension table

In [22]:
#Calling the dimension functions
df_resolution = dimension_tbl_fun(df[['Resolution']].copy())
#setting the order of the columns in dataframe
df_resolution = df_resolution[['Id', 'Resolution']]

In [23]:
#print of the sample of the dataframe
df_resolution

,Id,Resolution
0,0,Open or Active
1,1,Cite or Arrest Adult
2,2,Exceptional Adult
3,3,Unfounded


### Neighborhood Dimension table

In [24]:
#Calling the dimension functions
df_neighborhood = dimension_tbl_fun(df[['Analysis_Neighborhood']].copy())
#setting the order of the columns in dataframe
df_neighborhood = df_neighborhood[['Id', 'Analysis_Neighborhood']]
#renaming the column
df_neighborhood.rename(columns={'Analysis_Neighborhood': 'Neighborhood'}, inplace=True)

In [25]:
#print of the sample of the dataframe
df_neighborhood.head()

,Id,Neighborhood
0,0,Golden Gate Park
1,1,Financial District/South Beach
2,2,Potrero Hill
3,3,Tenderloin
4,4,Pacific Heights


### Location Dimension table

In [26]:
#Calling the dimension functions
df_location = dimension_tbl_fun(df[['Latitude','Longitude','Point']].copy())
#setting the order of the columns in dataframe
df_location = df_location[['Id', 'Latitude', 'Longitude', 'Point']]

In [27]:
#print of the sample of the dataframe
df_location.head()

,Id,Latitude,Longitude,Point
0,0,37.772895,-122.454285,POINT (-122.45428511766733 37.772895177200766)
1,1,37.787359,-122.408227,POINT (-122.40822672700406 37.78735926098589)
2,2,37.762290,-122.401324,POINT (-122.40132418490647 37.76228996810526)
3,3,37.787038,-122.418271,POINT (-122.41827098126804 37.787037946181535)
4,4,37.793977,-122.429804,POINT (-122.42980398313114 37.79397724418211)


### Intersection Dimension table

In [28]:
#Calling the dimension functions
df_intersection = dimension_tbl_fun(df[['Intersection']].copy())
#setting the order of the columns in dataframe
df_intersection = df_intersection[['Id', 'Intersection']]

### Crime Fact Table

In [29]:
#Init of a dataframe for the Fact database table
df_fact_table = df.copy()

#mapping the id(primary key) of dimension tables to the fact table
df_fact_table['Incident_Time_Id'] = df_fact_table['Incident_Datetime'].dt.time.map(df_time.set_index('Time')['Id'])
df_fact_table['Incident_Date_Id'] = df_fact_table['Incident_Datetime'].dt.date.map(df_date.set_index('Date')['Id'])
df_fact_table['Report_Time_Id'] = df_fact_table['Report_Datetime'].dt.time.map(df_time.set_index('Time')['Id'])
df_fact_table['Report_Date_Id'] = df_fact_table['Report_Datetime'].dt.date.map(df_date.set_index('Date')['Id'])
df_fact_table['Intersection_Id'] = df_fact_table['Intersection'].map(df_intersection.set_index('Intersection')['Id'])
df_fact_table['Police_district_Id'] = df_fact_table['Police_District'].map(df_police_district.set_index('Police_District')['Id'])
df_fact_table['Resolution_Id'] = df_fact_table['Resolution'].map(df_resolution.set_index('Resolution')['Id'])
df_fact_table['Neighborhood_Id'] = df_fact_table['Analysis_Neighborhood'].map(df_neighborhood.set_index('Neighborhood')['Id'])
#to merge fact and dimension table for the location to get id(primary key) to the fact table
df_fact_table = df_fact_table.merge(df_location, on=['Latitude', 'Longitude', 'Point'], how='left')
#renaming the column
df_fact_table.rename(columns={'Id':'Location_Id'}, inplace=True)

#dropping the unwanted columns
df_fact_table.drop(columns=['CAD_Number', 'CNN', 'Invest_In_Neighborhoods_(IIN)_Areas', 'ESNCAG_-_Boundary_File', 'Central_Market/Tenderloin_Boundary_Polygon_-_Updated', 'Civic_Center_Harm_Reduction_Project_Boundary', 
                 'HSOC_Zones_as_of_2018-06-05', 'Supervisor_District', 'Supervisor_District_2012', 'Latitude', 'Latitude', 'Point', 'Row_ID', 'Incident_Datetime', 'Incident_Date', 'Incident_Time', 'Incident_Year',
                 'Incident_Day_of_Week', 'Report_Datetime', 'Report_Type_Description', 'Incident_Category', 'Incident_Subcategory', 'Incident_Description', 'Resolution','Intersection', 'Police_District', 'Analysis_Neighborhood',
                 'Longitude', 'Neighborhoods', 'Current_Supervisor_Districts', 'Current_Police_Districts'], inplace=True)

#Dropping the NA values
df_fact_table.dropna(inplace=True)

#adding columns for the primary key
df_fact_table.reset_index(inplace=True)
df_fact_table.drop(columns=['index'], inplace=True)
df_fact_table['Id'] = df_fact_table.index


#to order the columns in fact database
df_fact_table = df_fact_table[['Id', 'Incident_ID', 'Incident_Number', 'Incident_Date_Id', 'Incident_Time_Id', 'Report_Date_Id', 'Report_Time_Id', 'Incident_Code', 'Report_Type_Code', 'Filed_Online', 'Police_district_Id', 'Resolution_Id',
                               'Location_Id', 'Neighborhood_Id', 'Intersection_Id']]

#converting the column id's to integer type|
df_fact_table[['Location_Id', 'Neighborhood_Id', 'Intersection_Id']] = df_fact_table[['Location_Id', 'Neighborhood_Id', 'Intersection_Id']].astype(int)

In [30]:
#print of the sample of the dataframe
df_fact_table.head()

,Id,Incident_ID,Incident_Number,Incident_Date_Id,Incident_Time_Id,Report_Date_Id,Report_Time_Id,Incident_Code,Report_Type_Code,Filed_Online,Police_district_Id,Resolution_Id,Location_Id,Neighborhood_Id,Intersection_Id
0,0,1254318,230182844,1895,840,1899,681,4134,II,0,3,0,0,0,0
1,1,1254393,230184129,1638,720,1899,1040,71000,II,0,2,0,1,1,1
2,2,1254826,230187101,1900,1050,1900,1082,4134,II,0,4,0,2,2,2
3,3,1256563,230199764,1905,950,1905,961,51040,II,0,7,0,3,3,3
4,4,1062676,210537297,1329,580,1329,580,62071,II,0,7,0,4,4,4


In [31]:
#print the number of rows and columns
df_fact_table.shape

(765399, 15)

In [32]:
#check the type format of columns in dataframe
df_fact_table.dtypes

Id                     int64
Incident_ID            int64
Incident_Number        int64
Incident_Date_Id       int64
Incident_Time_Id       int64
Report_Date_Id         int64
Report_Time_Id         int64
Incident_Code          int64
Report_Type_Code      object
Filed_Online           int32
Police_district_Id     int64
Resolution_Id          int64
Location_Id            int32
Neighborhood_Id        int32
Intersection_Id        int32
dtype: object

In [33]:
"""
select f.Id, f.Incident_Id, f.Incident_Number, 
       dti.Date as Incident_Date, ti.Time as Incidnet_Time, 
       dtr.Date as Report_Date, tr.Time as Report_time,
       inc.Incident_Code, inc.Incident_Category, inc.Incident_Subcategory, inc.Incident_Description,
       rec.Report_Type_Description,
       f.Filed_Online,
       pd.Police_District,
       res.Resolution,
       loc.Latitude,loc.Longitude,loc.Point,
       nbh.Analysis_Neighborhood,
       itr.Intersection
from crime-analysis-399301.crime_us_dataengineering.crime_incidents_fact_table as f
JOIN crime-analysis-399301.crime_us_dataengineering.date_dim as dti
ON f.Incident_Date_Id = dti.Id
JOIN crime-analysis-399301.crime_us_dataengineering.time_dim as ti
On f.Incident_Time_Id = ti.Id
JOIN crime-analysis-399301.crime_us_dataengineering.date_dim as dtr
ON f.Report_Date_Id = dtr.Id
JOIN crime-analysis-399301.crime_us_dataengineering.time_dim as tr
On f.Report_Time_Id = tr.Id
JOIN crime-analysis-399301.crime_us_dataengineering.incident_info_dim as inc
ON f.Incident_Code = inc.Incident_Code
JOIN crime-analysis-399301.crime_us_dataengineering.report_info_dim as rec
ON f.Report_Type_Code = rec.Report_Type_Code
JOIN crime-analysis-399301.crime_us_dataengineering.police_district_dim as pd
ON f.Police_district_Id = pd.Police_district_Id
JOIN crime-analysis-399301.crime_us_dataengineering.resolution_dim as res
On f.Resolution_Id = res.Resolution_Id
JOIN crime-analysis-399301.crime_us_dataengineering.location_dim as loc
ON f.Location_Id = loc.Location_Id
JOIN crime-analysis-399301.crime_us_dataengineering.neighborhood_dim as nbh
ON f.Neighborhood_Id = nbh.Neighborhood_Id
JOIN crime-analysis-399301.crime_us_dataengineering.intersection_dim as itr
ON f.Intersection_Id = itr.Intersection_Id
"""

'\nselect f.Id, f.Incident_Id, f.Incident_Number, \n       dti.Date as Incident_Date, ti.Time as Incidnet_Time, \n       dtr.Date as Report_Date, tr.Time as Report_time,\n       inc.Incident_Code, inc.Incident_Category, inc.Incident_Subcategory, inc.Incident_Description,\n       rec.Report_Type_Description,\n       f.Filed_Online,\n       pd.Police_District,\n       res.Resolution,\n       loc.Latitude,loc.Longitude,loc.Point,\n       nbh.Analysis_Neighborhood,\n       itr.Intersection\nfrom crime-analysis-399301.crime_us_dataengineering.crime_incidents_fact_table as f\nJOIN crime-analysis-399301.crime_us_dataengineering.date_dim as dti\nON f.Incident_Date_Id = dti.Id\nJOIN crime-analysis-399301.crime_us_dataengineering.time_dim as ti\nOn f.Incident_Time_Id = ti.Id\nJOIN crime-analysis-399301.crime_us_dataengineering.date_dim as dtr\nON f.Report_Date_Id = dtr.Id\nJOIN crime-analysis-399301.crime_us_dataengineering.time_dim as tr\nOn f.Report_Time_Id = tr.Id\nJOIN crime-analysis-399301.

## Unemployment Data

In [34]:
#importing the dataset
df_unemployment = pd.read_csv("../Data/Unemployment_data/Combined/Local_Area_Unemployment_Statistics__LAUS_.csv")

In [35]:
#Changing names of columns to insert Underscore in spaces and strip the blank space in the end
df_unemployment.rename(columns=lambda x: x.strip().replace(" ","_"),inplace=True)
#drop unnecessary columns
df_unemployment.drop(columns=['Status_(Preliminary_/_Final)', 'Month'], inplace=True)
#Convert the format of date column to datetime
df_unemployment['Date'] = pd.to_datetime(df_unemployment['Date'])

### Area Code Dimension table

In [36]:
#Init of a dict for area code-name
area_code_dict = {'Area_code' : ['0', '1'],
                  'Area_Name' : ['San Francisco County', 'California']}
#Converting dict to dataframe
df_area_code = pd.DataFrame.from_dict(area_code_dict)
#print the dataframe
df_area_code

,Area_code,Area_Name
0,0,San Francisco County
1,1,California


### Unemployment Fact Table

In [37]:
#fitler the unemployment dataframe for San Francisco County Area
df_county_unemployment  = df_unemployment[((df_unemployment['Area_Name'] == 'San Francisco County') | ((df_unemployment['Area_Name'] == 'California') & (df_unemployment['Seasonally_Adjusted_(Y/N)'] == 'Y'))) & (df_unemployment['Year'] >= 2018)].copy()
#reset the index of the fitlered dataframe
df_county_unemployment.reset_index(inplace=True)
#mapping the id(primary key) of dimension tables to the fact table
df_county_unemployment['Date_Id'] = df_county_unemployment['Date'].map(df_date.set_index('Date')['Id'])
#mapping the area name from the dimension table
df_county_unemployment['Area_code'] = df_county_unemployment['Area_Name'].map(df_area_code.set_index('Area_Name')['Area_code'])
#drop unnecessary columns in the dataframe
df_county_unemployment.drop(columns=['index', 'Area_Type', 'Year', 'Date', 'Area_Name', 'Seasonally_Adjusted_(Y/N)'], inplace=True)
#setting the order of the column names
df_county_unemployment = df_county_unemployment[['Date_Id', 'Area_code', 'Labor_Force', 'Employment', 'Unemployment', 'Unemployment_Rate']]
#print of the sample of the dataframe
df_county_unemployment.head()

,Date_Id,Area_code,Labor_Force,Employment,Unemployment,Unemployment_Rate
0,0,1,19197000,18352700,844200,0.044
1,0,0,558700,543200,15500,0.028
2,365,1,19367100,18527600,839500,0.043
3,365,0,574300,558900,15400,0.027
4,730,1,19590500,18754300,836200,0.043


#fitler the unemployment dataframe for San Francisco County Area
df_state_unemployment  = df_unemployment[(df_unemployment['Area_Name'] == 'California') & (df_unemployment['Year'] >= 2018)].copy()
#reset the index of the fitlered dataframe
df_state_unemployment.reset_index(inplace=True)
#mapping the id(primary key) of dimension tables to the fact table
df_state_unemployment['Date_Id'] = df_state_unemployment['Date'].map(df_date.set_index('Date')['Id'])
#mapping the area name from the dimension table
df_state_unemployment['Area_code'] = df_state_unemployment['Area_Name'].map(df_area_code.set_index('Area_Name')['Area_code'])
#drop unnecessary columns in the dataframe
df_state_unemployment.drop(columns=['index', 'Area_Type', 'Year', 'Date', 'Area_Name'], inplace=True)
#setting the order of the column names
df_state_unemployment = df_state_unemployment[['Date_Id', 'Area_code', 'Labor_Force', 'Employment', 'Unemployment', 'Unemployment_Rate']]
#print of the sample of the dataframe
df_state_unemployment.head()

## Weather Data

In [38]:
#importing the dataset
df_weather = pd.read_csv("../Data\Weather_data\POWER_Point_Daily_20180101_20231231_037d7748N_122d4194W_LST.csv")

In [39]:
df_weather.head()

,YEAR,MO,DY,T2MDEW,T2M,PRECTOTCORR,RH2M,QV2M,WS10M
0,2018,1,1,7.21,11.98,0.12,75.38,6.35,1.28
1,2018,1,2,6.79,11.11,0.00,75.25,6.10,2.95
2,2018,1,3,8.13,10.29,7.44,86.81,6.77,3.52
3,2018,1,4,12.28,13.08,3.63,95.00,8.91,3.41
4,2018,1,5,12.02,12.80,5.69,94.88,8.73,3.20


### Weather data Fact Table

In [40]:
df_weather.drop(df_weather.index[-1], inplace=True)
df_weather['Date'] = df_weather['YEAR'].astype(str) + '-' + df_weather['MO'].astype(str) + '-' + df_weather['DY'].astype(str)
df_weather['Date'] = pd.to_datetime(df_weather['Date'])
#mapping the id(primary key) of dimension tables to the fact table
df_weather['Date_Id'] = df_weather['Date'].map(df_date.set_index('Date')['Id'])
#drop unnecessary columns
df_weather.drop(columns=['YEAR', 'MO', 'DY', 'Date'], inplace=True)
#set the order of the columns in dataframe
df_weather = df_weather[['Date_Id', 'T2M', 'T2MDEW', 'PRECTOTCORR', 'RH2M', 'QV2M', 'WS10M']]
# renaming the columns
df_weather.columns = ['Date_Id', 'Temperature_Celcious', 'Dew_Frost_Celcious', 'Precipitation_mm/day', 'Relative_Humidity_Percentagte', 'Specific_Humidity_g/kg', 'Wind_Speed_m/s']

In [41]:
df_weather.head()

,Date_Id,Temperature_Celcious,Dew_Frost_Celcious,Precipitation_mm/day,Relative_Humidity_Percentagte,Specific_Humidity_g/kg,Wind_Speed_m/s
0,0,11.98,7.21,0.12,75.38,6.35,1.28
1,1,11.11,6.79,0.00,75.25,6.10,2.95
2,2,10.29,8.13,7.44,86.81,6.77,3.52
3,3,13.08,12.28,3.63,95.00,8.91,3.41
4,4,12.80,12.02,5.69,94.88,8.73,3.20


NASA/POWER CERES/MERRA2 Native Resolution Daily Data 

Dates (month/day/year): 01/01/2018 through 12/31/2023 

Location: Latitude  37.7623   Longitude -122.4506 

Elevation from MERRA-2: Average for 0.5 x 0.625 degree lat/lon region = 86.52 meters

The value for missing source data that cannot be computed or is outside of the sources availability range: 

-999 

Parameter(s): 

T2M             MERRA-2 Temperature at 2 Meters (C) 

TS              MERRA-2 Earth Skin Temperature (C) 

RH2M            MERRA-2 Relative Humidity at 2 Meters (%) 

QV2M            MERRA-2 Specific Humidity at 2 Meters (g/kg) 

PS              MERRA-2 Surface Pressure (kPa) 

WS10M           MERRA-2 Wind Speed at 10 Meters (m/s) 

PRECTOTCORR     MERRA-2 Precipitation Corrected (mm/day) 